In [1]:
import torch, pickle, argparse
import numpy as np, pytorch_lightning as pl, matplotlib.pyplot as plt, eagerpy as ep
from old_S2CNN_model import S2ConvNet
from data_loader import load_train_data, load_test_data
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD

In [2]:
filename = 'S2CNN_first_tests_smaller_training_sets.pickle'

with open(filename, 'rb') as file:
    loaded = pickle.load(file)
    
data = np.array(loaded)
hparams = argparse.Namespace(**data[1][1][1])

In [3]:
TEST_PATH = "s2_mnist.gz"
TRAIN_PATH = "s2_mnist_train_dwr_20000.gz"

train_data = load_train_data(TRAIN_PATH)
test_data = load_test_data(TEST_PATH)

In [4]:
model = S2ConvNet(hparams, train_data, test_data).eval()
trainer = pl.Trainer(gpus=1, logger=False)
best_model = torch.load('checkpoints/epoch=17_v14.ckpt')
model.load_state_dict(best_model['state_dict'])

# trainer.test(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


<All keys matched successfully>

In [5]:
fmodel = PyTorchModel(model, bounds=(0, 255))

In [6]:
images = test_data[:20][0].cuda()
labels = test_data[:20][1].cuda()
images = ep.astensor(images)
labels = ep.astensor(labels)

In [7]:
clean_acc = accuracy(fmodel, images, labels)
print(f"clean accuracy:  {clean_acc * 100:.1f} %")

load 0.pkl.gz... done
load 2.pkl.gz... done
load 1.pkl.gz... done
load 2.pkl.gz... done
load 8.pkl.gz... done
load 3.pkl.gz... done
load 4.pkl.gz... done
load 9.pkl.gz... done
load 14.pkl.gz... done
load 10.pkl.gz... done
load 15.pkl.gz... done
load 16.pkl.gz... done
load 11.pkl.gz... done
load 17.pkl.gz... done
load 12.pkl.gz... done
load 18.pkl.gz... done
load 19.pkl.gz... done
load 13.pkl.gz... done
load 20.pkl.gz... done
clean accuracy:  100.0 %


In [8]:
attack = LinfPGD()
epsilons = np.array([
    0.0,
    0.0002,
    0.0005,
    0.0008,
    0.001,
    0.0015,
    0.002,
    0.003,
    0.01,
    0.1,
    0.3,
    0.5,
    1,
])*255
raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilons)

In [9]:
robust_accuracy = 1 - success.float32().mean(axis=-1)
print("robust accuracy for perturbations with")
for eps, acc in zip(epsilons, robust_accuracy):
    print(f"  Linf norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

robust accuracy for perturbations with
  Linf norm ≤ 0.0   : 100.0 %
  Linf norm ≤ 0.051000000000000004: 100.0 %
  Linf norm ≤ 0.1275: 100.0 %
  Linf norm ≤ 0.20400000000000001: 100.0 %
  Linf norm ≤ 0.255 : 100.0 %
  Linf norm ≤ 0.3825: 100.0 %
  Linf norm ≤ 0.51  : 100.0 %
  Linf norm ≤ 0.765 : 100.0 %
  Linf norm ≤ 2.5500000000000003: 90.0 %
  Linf norm ≤ 25.5  :  0.0 %
  Linf norm ≤ 76.5  :  0.0 %
  Linf norm ≤ 127.5 :  0.0 %
  Linf norm ≤ 255.0 :  0.0 %
